# 입문자를 위한, 파이썬/R 데이터 분석      

## Part(1) - Day07  :  

]

## Today's mission

- 데이터 수집 (다양한 팁)
   - 프레임셋 전환
   - 크롤링 태그 찾기
   - 팝업창 제어
- 실습 (1) Web crawling test under the tag of framset
- 실습 (2) Tag 찾아보기 (예시) : 공지사항 내용 
- 실습 (3) Web crawling test for popup


- (공공데이터분석) 국토부 아파트매매 실거래가 API[data.go.kr] 연동

]

##  (공공데이터분석) 국토부 아파트매매 실거래가 API[data.go.kr] 연동

#### XML 국토교통부_아파트매매 실거래자료 : https://www.data.go.kr/data/15058747/openapi.do

#### 행정표준관리시스템 : https://www.code.go.kr/index.do ( [메뉴] 자주이용하는 코드 > 법정동 )

]

### 1. Import Packages

In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

### 2. Data Loading 

#### : API Key Setting

In [60]:
f=open("data/my_api_key.txt","r")
lines=f.readlines()
my_key=lines[0]
f.close()

In [61]:
my_key

'c%2B9O5FYgyH4YJA4VAeems4Nhk34TQIIAYm1ocLLGy%2FrG02SYK2k8AU0QBUgtn2q2CZC4CDxYkay1aM9GQtw6wg%3D%3D'

#### : Pre-defined

In [62]:
#"http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade
#        ?
#        LAWD_CD=11110
#        &
#        DEAL_YMD=201512
#        &
#        serviceKey=서비스키"

In [63]:
base_url = "http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade"

In [64]:
area_cd = "LAWD_CD=" + "11680"    # 11680 : 강남구
deal_dt = "DEAL_YMD=" + "202201"
svc_key = "serviceKey=" + my_key

In [67]:
url = base_url + "?" + area_cd + "&" + deal_dt + "&" + svc_key
url

'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?LAWD_CD=11680&DEAL_YMD=202201&serviceKey=c%2B9O5FYgyH4YJA4VAeems4Nhk34TQIIAYm1ocLLGy%2FrG02SYK2k8AU0QBUgtn2q2CZC4CDxYkay1aM9GQtw6wg%3D%3D'

#### : Request Test

In [68]:
response = requests.get(url)
response

<Response [200]>

In [69]:
response.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><거래금액>      76,000</거래금액><거래유형>중개거래</거래유형><건축년도>2002</건축년도><년>2022</년><법정동> 역삼동</법정동><아파트>갤러리아트</아파트><월>1</월><일>3</일><전용면적>80.39</전용면적><중개사소재지>서울 강남구</중개사소재지><지번>783-28</지번><지역코드>11680</지역코드><층>5</층><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item><item><거래금액>     325,000</거래금액><거래유형>중개거래</거래유형><건축년도>2006</건축년도><년>2022</년><법정동> 역삼동</법정동><아파트>개나리래미안</아파트><월>1</월><일>3</일><전용면적>129.8</전용면적><중개사소재지>서울 강남구</중개사소재지><지번>754</지번><지역코드>11680</지역코드><층>14</층><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item><item><거래금액>      20,500</거래금액><거래유형>중개거래</거래유형><건축년도>2004</건축년도><년>2022</년><법정동> 역삼동</법정동><아파트>강남역우정에쉐르</아파트><월>1</월><일>5</일><전용면적>17.13</전용면적><중개사소재지>서울 강남구</중개사소재지><지번>823-7</지번><지역코드>11680</지역코드><층>11</층><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item><item><거래금액>      58,000</거래금액><거래유형>중개거래</거래유형><건축년도>2004</건축년도><년>2022</년><법정동> 역삼동</법정동><아파트>우정에

### Addtional Preparation : 

#### -1)  Input Data : lawd_cd / 법정동 코드(지역코드) 가공

In [70]:
lawd_cd = pd.read_csv("data/법정동코드_전체자료.txt", sep='\t', encoding ='cp949') #'euc-kr', 'utf-8'
lawd_cd

법정동코드                  법정동명 폐지여부
0      1100000000                 서울특별시   존재
1      1111000000             서울특별시 종로구   존재
2      1111010100         서울특별시 종로구 청운동   존재
3      1111010200         서울특별시 종로구 신교동   존재
4      1111010300         서울특별시 종로구 궁정동   존재
...           ...                   ...  ...
46287  5013032022  제주특별자치도 서귀포시 표선면 하천리   존재
46288  5013032023  제주특별자치도 서귀포시 표선면 성읍리   존재
46289  5013032024  제주특별자치도 서귀포시 표선면 가시리   존재
46290  5013032025  제주특별자치도 서귀포시 표선면 세화리   존재
46291  5013032026  제주특별자치도 서귀포시 표선면 토산리   존재

[46292 rows x 3 columns]

In [71]:
# Column 이름 변경 : 데이터 조작 편의성을 위함
print(lawd_cd.columns)

Index(['법정동코드', '법정동명', '폐지여부'], dtype='object')


In [72]:
lawd_cd.columns = ["code", "name", "check"]
lawd_cd

code                  name check
0      1100000000                 서울특별시    존재
1      1111000000             서울특별시 종로구    존재
2      1111010100         서울특별시 종로구 청운동    존재
3      1111010200         서울특별시 종로구 신교동    존재
4      1111010300         서울특별시 종로구 궁정동    존재
...           ...                   ...   ...
46287  5013032022  제주특별자치도 서귀포시 표선면 하천리    존재
46288  5013032023  제주특별자치도 서귀포시 표선면 성읍리    존재
46289  5013032024  제주특별자치도 서귀포시 표선면 가시리    존재
46290  5013032025  제주특별자치도 서귀포시 표선면 세화리    존재
46291  5013032026  제주특별자치도 서귀포시 표선면 토산리    존재

[46292 rows x 3 columns]

In [73]:
lawd_cd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46292 entries, 0 to 46291
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   code    46292 non-null  int64 
 1   name    46292 non-null  object
 2   check   46292 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [74]:
in_name = "종로구"
lawd_cd[(lawd_cd['name'].str.contains(in_name)) & (lawd_cd['check'] == "존재")]

code           name check
1   1111000000      서울특별시 종로구    존재
2   1111010100  서울특별시 종로구 청운동    존재
3   1111010200  서울특별시 종로구 신교동    존재
4   1111010300  서울특별시 종로구 궁정동    존재
5   1111010400  서울특별시 종로구 효자동    존재
..         ...            ...   ...
84  1111018300  서울특별시 종로구 평창동    존재
85  1111018400  서울특별시 종로구 부암동    존재
86  1111018500  서울특별시 종로구 홍지동    존재
87  1111018600  서울특별시 종로구 신영동    존재
88  1111018700  서울특별시 종로구 무악동    존재

[88 rows x 3 columns]

In [75]:
tmp = lawd_cd[(lawd_cd['name'].str.contains(in_name)) & (lawd_cd['check'] == "존재")].head(1)
tmp

code       name check
1  1111000000  서울특별시 종로구    존재

In [76]:
tmp.loc[1,'code']

1111000000

In [77]:
tmp.iloc[0,0]

1111000000

In [78]:
tmp = tmp.iloc[0,0]
#str(tmp)
tmp = tmp.astype(str)
tmp[0:5]

'11110'

In [79]:
lawd_cd[(lawd_cd['name'].str.contains(in_name)) & (lawd_cd['check'] == "존재")].head(1).iloc[0,0].astype(str)[0:5]

'11110'

In [80]:
# Function 작성 :
def find_lawd_cd(in_text):
    return lawd_cd[(lawd_cd['name'].str.contains(in_text)) & (lawd_cd['check'] == "존재")].head(1).iloc[0,0].astype(str)[0:5]    

In [81]:
in_name = "종로구"
find_lawd_cd(in_name)

'11110'

#### -2)  Input Data : lawd_cd, deal_ymd 입력받기

In [82]:
in_lawd = input("검색지역 입력 : ")

검색지역 입력 : 


In [83]:
in_deal_ymd = input("계약월 입력(예: 201912) : ")

계약월 입력(예: 201912) : 


In [84]:
in_svckey = "fq448BTENSGVmu47zvzjDudVK1LUOPdIONYmOk774uKTYukGYTIZdMjVIURNLrwHFbDrRgCZRXehyB%2FR2YxQ5A%3D%3D"

In [85]:
in_lawd_cd = find_lawd_cd(in_lawd)
in_lawd_cd

'11000'

In [86]:
area_cd = "LAWD_CD=" + in_lawd_cd
deal_dt = "DEAL_YMD=" + in_deal_ymd
svc_key = "serviceKey=" + in_svckey

In [87]:
url = base_url + "?" + area_cd + "&" + deal_dt + "&" + svc_key
url

'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?LAWD_CD=11000&DEAL_YMD=&serviceKey=fq448BTENSGVmu47zvzjDudVK1LUOPdIONYmOk774uKTYukGYTIZdMjVIURNLrwHFbDrRgCZRXehyB%2FR2YxQ5A%3D%3D'

In [88]:
response = requests.get(url)
response

<Response [200]>

### 3. Data Preprocessng 

#### : web data / API output

In [89]:
soup = BeautifulSoup(response.text, "lxml")
soup

C:\Users\salix\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


<?xml version="1.0" encoding="UTF-8" standalone="yes"?><html><body><response><header><resultcode>99</resultcode><resultmsg>SERVICE KEY IS NOT REGISTERED ERROR.</resultmsg></header></response></body></html>

In [90]:
soup.find_all('item')

[]

In [91]:
import xml.etree.ElementTree as ET

def find_xml_items(response):
    root = ET.fromstring(response.content)
    item_list = []
    for child in root.find('body').find('items'):
        elements = child.findall('*')
        data = {}
        for element in elements:
            tag = element.tag.strip()
            text = element.text.strip()
            # print tag, text
            data[tag] = text
        item_list.append(data)  
    return item_list

In [92]:
items_list = find_xml_items(response)
items_list

AttributeError: 'NoneType' object has no attribute 'find'

### 4. Result : Display & Save as file 

In [93]:
result = pd.DataFrame(items_list)
print(result.shape)
result.head()

NameError: name 'items_list' is not defined

#### : file name

In [94]:
name_info = in_lawd + "_" + in_deal_ymd
name_info

'_'

In [95]:
file_name = f"data/result_{name_info}.csv"
file_name

'data/result__.csv'

In [96]:
result.to_csv(file_name, index=False)

NameError: name 'result' is not defined

## Appendix : Function 

In [107]:
def deal_by_lawd(cd, ymd):
    
    svckey = my_key
    
    area_cd = "LAWD_CD=" + cd
    deal_dt = "DEAL_YMD=" + ymd
    svc_key = "serviceKey=" + svckey
    url = base_url + "?" + area_cd + "&" + deal_dt + "&" + svc_key

    response = requests.get(url)
    items_list = find_xml_items(response)
    result = pd.DataFrame(items_list)
    
    name_info = in_lawd + "_" + in_deal_ymd
    file_name = f"data/result_{name_info}.csv"

    result.to_csv(file_name, index=False)
    
    return file_name

In [108]:
#### - Input Data : LAWD_CD, DEAL_YMD, serviceKey  입력받기
in_lawd = input("검색지역 입력 : ")
print(in_lawd)

in_lawd_cd = find_lawd_cd(in_lawd)
print(in_lawd_cd)

in_deal_ymd = input("계약월 입력(예: 201911) : ")
print(in_deal_ymd)

검색지역 입력 : 중구
중구
11140
계약월 입력(예: 201911) : 202304
202304


In [109]:
deal_by_lawd(in_lawd_cd, in_deal_ymd)

'data/result_중구_202304.csv'